In [1]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# CRF7.ipynb
#
# (1) Token lemmatization using Zemberek TurkishMorphology  (DO NOT lemmatize tokens starting with uppercase letter)
# (2) Token features:
#     (a) token
#     (b) is_first           : is token at the beginning of the sentence?
#     (c) is_last            : is token at the end of the sentence?
#     (d) is_capitalized     : does token start with a capital letter? 
#     (e) is_all_capitalized : is all letters of the token capitalized?
#     (f) is_capitals_inside : is there any capitalized letter inside the token?
#     (g) is_numeric         : is token numeric?
#     (h) prefix-1           : first letter of the token
#     (i) suffix-1           : last letter of the token
#     (j) prefix-2           : first two letters of the token
#     (k) suffix-2           : last two letters of the token
#     (l) prefix-3           : first three letters of the token
#     (m) suffix-3           : last three letters of the token
#     (n) next-token         : following token
#     (o) prev-token         : preceding token
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# May, 2019
# --------------------------------------------------

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from seqeval.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn_crfsuite import CRF
import jpype as jp
import pickle
import scipy

In [3]:
ZEMBEREK_PATH = 'bin/zemberek-full.jar'

# Start the JVM
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [4]:
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')
morphology        = TurkishMorphology.createWithDefaults()

In [5]:
def readFile(filepath):

    text     = []
    sentence = []

    with open(filepath, 'r') as infile:
        for line in infile:
            word, _, _, _ = line.strip().split('\t')

            if word == '<S>':
                text.append(sentence)
                sentence = []
                continue

            sentence.append(line.strip())

    return text

In [6]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt')

In [7]:
def getFeature(token, token_index, sentence):

    feature = {'token'             : token,
               'is_first'          : token_index == 0,
               'is_last'           : token_index == len(sentence) - 1,
               
               'is_capitalized'    : token[0].upper() == token[0],
               'is_all_capitalized': token.upper() == token,
               'is_capitals_inside': token[1:].lower() != token[1:],
               'is_numeric'        : token.isdigit(),
               
               'prefix-1'          : token[0],
               'suffix-1'          : token[-1],
               
               'prefix-2'          : '' if len(token) < 2  else token[:2],
               'suffix-2'          : '' if len(token) < 2  else token[-2:],
               
               'prefix-3'          : '' if len(token) < 3  else token[:3],
               'suffix-3'          : '' if len(token) < 3  else token[-3:],
               
               'prev-token'        : '' if token_index == 0 else sentence[token_index - 1][0],
               'next-token'        : '' if token_index == len(sentence) - 1 else sentence[token_index + 1][0],
              }
    
    return feature

In [8]:
trainFeatures = []
trainLabels   = []

for sentence in trainText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    trainFeatures.append(features)
    trainLabels.append(labels)

In [9]:
validFeatures = []
validLabels   = []

for sentence in validText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    validFeatures.append(features)
    validLabels.append(labels)

In [10]:
testFeatures = []
testLabels   = []

for sentence in testText:
    SENT     = []
    features = []
    labels   = []
    for token in sentence:
        word, _, _, label = token.split('\t')
        
        if word[0] == word[0].upper():
            root = word
        else:
            results = morphology.analyze(word).analysisResults
            root    = results[0].getLemmas()[0] if results else word
        SENT.append(root)
        labels.append(label)
        
    for i, word in enumerate(SENT):
        features.append(getFeature(word, i, SENT))
    
    testFeatures.append(features)
    testLabels.append(labels)

In [11]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [12]:
# define fixed parameters and parameters to search
crf = CRF(  algorithm='lbfgs',
            max_iterations=100,
            all_possible_transitions=True,
            verbose=True)

params_space = {'c1': scipy.stats.expon(scale=0.5),
                'c2': scipy.stats.expon(scale=0.05)}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        random_state=123,
                        scoring=f1_scorer)

rs.fit(trainvalFeatures, trainvalLabels)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/cetinsamet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 45.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 161.2min finished
loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:05<00:00, 6398.61it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 65615
Seconds required: 1.289

L-BFGS optimization
c1: 0.128642
c2: 0.040072
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.12  loss=764334.27 active=65300 feature_norm=1.00
Iter 2   time=1.74  loss=527749.14 active=64852 feature_norm=6.28
Iter 3   time=0.59  loss=416419.56 active=61464 feature_norm=5.37
Iter 4   time=3.54  loss=251852.06 active=62942 feature_norm=3.94
Iter 5   time=1.77  loss=232081.03 active=64546 feature_norm=3.76
Iter 6   time=0.57  loss=213047.20 active=64956 feature_norm=4.23
Iter 7   time=0.57  loss=180184.87 active=64471 feature_norm=5.58
Iter 8   time=0.57  loss=146714.05 active=60869 feature_norm=8.47
Iter 9   time=0.57  loss=134546.59 active=61444 feature_norm=9.91
Iter 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...ne,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True),
          fit_params=None, iid='warn', n_iter=50, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1afba58a58>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11128f358>},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn',
          scoring=make_scorer(flat_f1_score, average=weighted), verbose=1)

In [13]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.1286420400585254, 'c2': 0.040071684090734375}
best CV score: 0.9680107799007583
model size: 1.71M


In [14]:
crf = rs.best_estimator_
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.1286420400585254, c2=0.040071684090734375,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [15]:
crf.fit(trainvalFeatures, trainvalLabels)

loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:07<00:00, 4414.05it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 65615
Seconds required: 1.532

L-BFGS optimization
c1: 0.128642
c2: 0.040072
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.23  loss=764334.27 active=65300 feature_norm=1.00
Iter 2   time=1.76  loss=527749.14 active=64852 feature_norm=6.28
Iter 3   time=0.60  loss=416419.56 active=61464 feature_norm=5.37
Iter 4   time=3.81  loss=251852.06 active=62942 feature_norm=3.94
Iter 5   time=1.86  loss=232081.03 active=64546 feature_norm=3.76
Iter 6   time=0.58  loss=213047.20 active=64956 feature_norm=4.23
Iter 7   time=0.58  loss=180184.87 active=64471 feature_norm=5.58
Iter 8   time=0.60  loss=146714.05 active=60869 feature_norm=8.47
Iter 9   time=0.58  loss=134546.59 active=61444 feature_norm=9.91
Iter 

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.1286420400585254, c2=0.040071684090734375,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [16]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf7.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("model is saved.")

model is saved.


In [17]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf7.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [18]:
trainvalPredLabels = crf.predict(trainvalFeatures)

print("### TRAINVAL CLASSIFICATION REPORT ###\n")
print(classification_report(trainvalLabels, trainvalPredLabels))

### TRAINVAL CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

    LOCATION       0.98      0.99      0.98      9409
ORGANIZATION       0.98      0.97      0.97      9034
        DATE       0.91      0.94      0.93      3103
      PERSON       0.99      0.98      0.99     14476
       MONEY       0.99      0.98      0.98       594
     PERCENT       1.00      1.00      1.00       617
        TIME       1.00      1.00      1.00       175

   micro avg       0.98      0.98      0.98     37408
   macro avg       0.98      0.98      0.98     37408



In [19]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

    LOCATION       0.88      0.84      0.86      1091
ORGANIZATION       0.83      0.78      0.81       862
      PERSON       0.90      0.84      0.87      1594
     PERCENT       0.98      0.92      0.95       107
        DATE       0.81      0.87      0.84       364
       MONEY       0.88      0.74      0.81       113
        TIME       0.90      0.78      0.84        23

   micro avg       0.88      0.83      0.85      4154
   macro avg       0.88      0.83      0.85      4154



In [20]:
# Shutting down the JVM
jp.shutdownJVM()